In [53]:
import pandas as pd
import re
import sys
import os
import re

In [54]:
data_oranda= pd.read_csv("../data/oranda_review.csv")
data_oranda.head()

,이름,작성일자,평점,리뷰 내용,만족도
0,솔직한깡지,2024.11.16,5,지난번 아이 생일때 선생님들 드리려고 샀다가몇개 먹어보고 맛있어서 재구매했어요 장바...,맛있어요
1,김오성,2024.11.11,5,우주오란다 플래인 강정 주문햇습니다 이거 반틈은 내가 먹고 반틈은 울아부지 드릴껍니...,맛있어요
2,김오성,2024.09.26,5,ㅈㅣ는 84세이신 울아부지랑 둘이서 살고 있습니다 울아부지 옛날에는 장승같이 커다란...,맛있어요
3,이*진,2024.09.22,4,《 우주오란다 》 * 용량 : 30g × 12개 * 소비기한 : 2025.04...,맛있어요
4,36참나리,2024.09.22,5,"클래식한 간식 중 최고봉, 오란다.달달한 간식이 생각나 주문했다.옛날 문방구에서 사...",맛있어요


In [55]:
data_oranda.shape

(1500, 5)

# 데이터 필터링

등록된 리뷰내용이 있는 것만 가져오기

In [56]:

filtered_data_oranda = data_oranda[data_oranda["리뷰 내용"] != "등록된 리뷰내용이 없습니다"]
filtered_data_oranda


,이름,작성일자,평점,리뷰 내용,만족도
0,솔직한깡지,2024.11.16,5,지난번 아이 생일때 선생님들 드리려고 샀다가몇개 먹어보고 맛있어서 재구매했어요 장바...,맛있어요
1,김오성,2024.11.11,5,우주오란다 플래인 강정 주문햇습니다 이거 반틈은 내가 먹고 반틈은 울아부지 드릴껍니...,맛있어요
2,김오성,2024.09.26,5,ㅈㅣ는 84세이신 울아부지랑 둘이서 살고 있습니다 울아부지 옛날에는 장승같이 커다란...,맛있어요
3,이*진,2024.09.22,4,《 우주오란다 》 * 용량 : 30g × 12개 * 소비기한 : 2025.04...,맛있어요
4,36참나리,2024.09.22,5,"클래식한 간식 중 최고봉, 오란다.달달한 간식이 생각나 주문했다.옛날 문방구에서 사...",맛있어요
...,...,...,...,...,...
837,정*운,2024.07.27,5,배송속도도 제품품질도 맛도 좋아요,맛있어요
840,박아름,2023.12.01,5,빠른배송에 너무 맛있어요!,예상보다 맛있어요
841,변*곤,2024.05.27,5,배송도 빠르고 파손없이 잘 수령하였습니다.,맛있어요
842,김*화,2023.12.07,5,배송도 빠르고~너무 맛잇어요~~~,예상보다 맛있어요


In [57]:
df = filtered_data_oranda

불용어 제거 & 특수문자 제거

In [58]:
from konlpy.tag import Kkma #형태소 분석기
from konlpy.tag import Okt

kkma = Kkma()
okt = Okt()

In [59]:
# sys.path.append(os.path.dirname('PyKoSpacing/'))
# from pykospacing import spacing #한글 문장에서 띄어쓰기 보정 
# spacing = Spacing()  # Spacing 객체 초기화
# pip install git+https://github.com/haven-jeon/PyKoSpacing.git

def preprocessing(review, filter=None):
    total_review = ''
    #인풋리뷰
    for idx in range(len(review)):
        r = review[idx]
        #하나의 리뷰에서 문장 단위로 자르기
        #불필요한 정보제거 name의 각 단어를 review에서 제거 
        # 이름이 주어졌을 경우에만 이름 정보 제거
        if filter:
            filter_parts = filter.split(' ')
            for part in filter_parts:
                r = re.sub(part, '', r)
        sentence = r
        #특수문자, 영어 알파벳, 초성/중성/종성(예: "ㄱ", "ㅏ" 등)을 제거.
        sentence = re.sub('\n','',sentence)
        sentence = re.sub('\u200b','',sentence)
        sentence = re.sub('\xa0','',sentence)
        sentence = re.sub('([a-zA-Z])','',sentence)
        sentence = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',sentence)
        sentence = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',sentence)
        #전처리 후 문장이 비어있으면 다음 리뷰로 넘어감 
        if len(sentence) == 0:
            continue
        #동사를 기본형으로 변환
        sentence = okt.pos(sentence, stem = True)
        word = []
        #명사가 아닌 경우 건너띔(list에 추가 안함함)
        for i in sentence:
            if not i[1] == 'Noun':
                continue
            if len(i[0]) == 1:
                continue
            word.append(i[0])
        #단어들을 공백으로 연결하여 하나의 문장으로 변환
        #문장 끝에 마침표 추가
        #전체 리뷰에 추가
        word = ' '.join(word)
        word += '. '
        total_review += word
    return total_review

In [60]:
# 전처리 수행 #21초 정도 걸리는듯듯
df['전처리된_리뷰'] = df.apply(lambda row: preprocessing([row['리뷰 내용']]), axis=1)

C:\Users\zin\AppData\Local\Temp\ipykernel_28696\447569505.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['전처리된_리뷰'] = df.apply(lambda row: preprocessing([row['리뷰 내용']]), axis=1)


In [61]:
df.head()

,이름,작성일자,평점,리뷰 내용,만족도,전처리된_리뷰
0,솔직한깡지,2024.11.16,5,지난번 아이 생일때 선생님들 드리려고 샀다가몇개 먹어보고 맛있어서 재구매했어요 장바...,맛있어요,지난번 아이 생일 선생님 몇개 보고 장바구니 종종 할인 행사 할인 전혀 장바구니 할...
1,김오성,2024.11.11,5,우주오란다 플래인 강정 주문햇습니다 이거 반틈은 내가 먹고 반틈은 울아부지 드릴껍니...,맛있어요,우주 래인 강정 햇습니 아부지 드릴 아부지 우주 래인 강정 옛날 이제 나이 무가 요...
2,김오성,2024.09.26,5,ㅈㅣ는 84세이신 울아부지랑 둘이서 살고 있습니다 울아부지 옛날에는 장승같이 커다란...,맛있어요,아부지 둘이서 아부지 옛날 장승 모습 는데 나이 가부장 잘못 뚜디리 기억 아부지 아...
3,이*진,2024.09.22,4,《 우주오란다 》 * 용량 : 30g × 12개 * 소비기한 : 2025.04...,맛있어요,우주 용량 소비 기한 구매 이유 물엿 코팅 달달 식감 과자 특유 냄새 추억 우주 쿠...
4,36참나리,2024.09.22,5,"클래식한 간식 중 최고봉, 오란다.달달한 간식이 생각나 주문했다.옛날 문방구에서 사...",맛있어요,클래식 간식 최고봉 달달 간식 주문 옛날 문방구 오란 입천장 이번 구매 수제 강정 ...


특정 단어 제거(이벤트, 체험단 등)

In [62]:
# def remove_rows_with_keywords(dataframe, column_name, keywords):
#     """
#     특정 단어가 포함된 문장을 제거하는 함수.

#     Args:
#         dataframe (pd.DataFrame): 데이터프레임
#         column_name (str): 처리할 열 이름
#         keywords (list): 제거할 단어 리스트

#     Returns:
#         pd.DataFrame: 특정 단어가 포함된 행이 제거된 데이터프레임
#     """
#     # 특정 단어가 포함된 행 제거
#     pattern = "|".join(keywords)  # 키워드 리스트를 정규식 패턴으로 변환
#     filtered_df = dataframe[~dataframe[column_name].str.contains(pattern, na=False)]
#     return filtered_df

# # 사용 예시
# keywords = ["이벤트", "체험단"]
# filtered_data_oranda = remove_rows_with_keywords(data_oranda, "리뷰 내용", keywords)

# # 결과 확인
# print(filtered_data_oranda)
# len(filtered_data_oranda)

Case1: 리뷰 별점 구분없이 키워드 뽑기 vs 리뷰 평점별로 키워드 뽑기

Case2: 문장 별로 나누어서 키워드 뽑기 vs 문장 분리 하지 말기
- 일단 문장 분리하면 키워드가 뽑히지 않는 경우도 있음(문장이 너무 짧아서)


Case3: 명사만 이용

https://chaeeunsong.tistory.com/26 참고고


키워드 후보군 DB 만들기 -> 키워드 선정?

# Model1: krwordrank & gensim

In [67]:
from krwordrank.sentence import summarize_with_sentences

# 데이터프레임 열을 처리하는 함수
def df_krwordRank(df, review_col, filter=None, top_k=5, num_keywords=5, num_keysents=1, full=True):
    """
    데이터프레임에서 특정 열을 전처리하고 키워드 추출 결과를 데이터프레임에 저장.

    Parameters:
    - df: pd.DataFrame - 입력 데이터프레임
    - review_col: str - 리뷰 내용을 포함하는 열 이름
    - filter: 필터링할 단어 혹은 문장장
    - top_k: int - 각 리뷰별 상위 키워드 개수
    - num_keywords: int - 추출할 키워드 개수
    - num_keysents: int - 추출할 중요 문장 개수
    - full: bool - True일 경우 모든 리뷰를 합쳐서 처리

    Returns:
    - result_df: pd.DataFrame - 키워드 열이 추가된 데이터프레임
    """
    # 새로운 열을 저장할 리스트
    krwordrank_results = []

    for idx, row in df.iterrows():
        # 리뷰 내용 및 이름 가져오기
        review_content = row[review_col]
        filter 
        # 리뷰 전처리
        st = ''
        for text in review_content.split('. '):  # 문장 단위 분리
            # text = preprocessing(text, name)
            st += text
        
        # 문장 리스트
        texts = st.split('. ')
        
        # Stopwords 설정 
        # try:
        #     stopwords = {name.split(' ')[0], name.split(' ')[1]}
        # except IndexError:
        #     stopwords = set()
        
        # 키워드 추출
        try:
            keywords, _ = summarize_with_sentences(
                texts, 
                stopwords=None,
                num_keywords=num_keywords, 
                num_keysents=num_keysents
            )
            # 상위 키워드 추출
            top_keywords = sorted(keywords.items(), key=lambda x: x[1], reverse=True)[:top_k]
            top_keywords = [word for word, _ in top_keywords]  # 키워드만 추출
        
        #문장이 너무 짧은 경우 키워드가 존재하지 않음(은근 기준이 빡센듯...?)
        except ValueError:
            print(f"[{idx}] 키워드가 없습니다. 스킵합니다.")
            top_keywords = None
        
        # 결과 저장
        krwordrank_results.append(top_keywords)

    # 새로운 열 추가
    df['krwordrank'] = krwordrank_results
    return df


In [68]:
df.head()

,이름,작성일자,평점,리뷰 내용,만족도,전처리된_리뷰,krwordrank
0,솔직한깡지,2024.11.16,5,지난번 아이 생일때 선생님들 드리려고 샀다가몇개 먹어보고 맛있어서 재구매했어요 장바...,맛있어요,지난번 아이 생일 선생님 몇개 보고 장바구니 종종 할인 행사 할인 전혀 장바구니 할...,[구매]
1,김오성,2024.11.11,5,우주오란다 플래인 강정 주문햇습니다 이거 반틈은 내가 먹고 반틈은 울아부지 드릴껍니...,맛있어요,우주 래인 강정 햇습니 아부지 드릴 아부지 우주 래인 강정 옛날 이제 나이 무가 요...,None
2,김오성,2024.09.26,5,ㅈㅣ는 84세이신 울아부지랑 둘이서 살고 있습니다 울아부지 옛날에는 장승같이 커다란...,맛있어요,아부지 둘이서 아부지 옛날 장승 모습 는데 나이 가부장 잘못 뚜디리 기억 아부지 아...,[아부]
3,이*진,2024.09.22,4,《 우주오란다 》 * 용량 : 30g × 12개 * 소비기한 : 2025.04...,맛있어요,우주 용량 소비 기한 구매 이유 물엿 코팅 달달 식감 과자 특유 냄새 추억 우주 쿠...,"[포장, 식감, 우주]"
4,36참나리,2024.09.22,5,"클래식한 간식 중 최고봉, 오란다.달달한 간식이 생각나 주문했다.옛날 문방구에서 사...",맛있어요,클래식 간식 최고봉 달달 간식 주문 옛날 문방구 오란 입천장 이번 구매 수제 강정 ...,"[네덜란, 오란]"


In [69]:
# 함수 호출
result_df = df_krwordRank(df, review_col='전처리된_리뷰', top_k=5)

[1] 키워드가 없습니다. 스킵합니다.
[5] 키워드가 없습니다. 스킵합니다.
[6] 키워드가 없습니다. 스킵합니다.
[7] 키워드가 없습니다. 스킵합니다.
[9] 키워드가 없습니다. 스킵합니다.
[10] 키워드가 없습니다. 스킵합니다.
[11] 키워드가 없습니다. 스킵합니다.
[14] 키워드가 없습니다. 스킵합니다.
[16] 키워드가 없습니다. 스킵합니다.
[18] 키워드가 없습니다. 스킵합니다.
[20] 키워드가 없습니다. 스킵합니다.
[21] 키워드가 없습니다. 스킵합니다.
[23] 키워드가 없습니다. 스킵합니다.
[26] 키워드가 없습니다. 스킵합니다.
[27] 키워드가 없습니다. 스킵합니다.
[28] 키워드가 없습니다. 스킵합니다.
[29] 키워드가 없습니다. 스킵합니다.
[30] 키워드가 없습니다. 스킵합니다.
[31] 키워드가 없습니다. 스킵합니다.
[32] 키워드가 없습니다. 스킵합니다.
[33] 키워드가 없습니다. 스킵합니다.
[34] 키워드가 없습니다. 스킵합니다.
[35] 키워드가 없습니다. 스킵합니다.
[36] 키워드가 없습니다. 스킵합니다.
[37] 키워드가 없습니다. 스킵합니다.
[38] 키워드가 없습니다. 스킵합니다.
[40] 키워드가 없습니다. 스킵합니다.
[41] 키워드가 없습니다. 스킵합니다.
[42] 키워드가 없습니다. 스킵합니다.
[43] 키워드가 없습니다. 스킵합니다.
[44] 키워드가 없습니다. 스킵합니다.
[45] 키워드가 없습니다. 스킵합니다.
[46] 키워드가 없습니다. 스킵합니다.
[48] 키워드가 없습니다. 스킵합니다.
[49] 키워드가 없습니다. 스킵합니다.
[50] 키워드가 없습니다. 스킵합니다.
[51] 키워드가 없습니다. 스킵합니다.
[52] 키워드가 없습니다. 스킵합니다.
[53] 키워드가 없습니다. 스킵합니다.
[54] 키워드가 없습니다. 스킵합니다.
[55] 키워드가 없습니다. 스킵합니다.
[56] 키워드가 없습니다. 스킵합니다.
[57] 키워드가 없습니다. 스킵합니다.
[58] 키워드가 없습니다. 

C:\Users\zin\AppData\Local\Temp\ipykernel_28696\3437492222.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['krwordrank'] = krwordrank_results


In [70]:
result_df.head()

,이름,작성일자,평점,리뷰 내용,만족도,전처리된_리뷰,krwordrank
0,솔직한깡지,2024.11.16,5,지난번 아이 생일때 선생님들 드리려고 샀다가몇개 먹어보고 맛있어서 재구매했어요 장바...,맛있어요,지난번 아이 생일 선생님 몇개 보고 장바구니 종종 할인 행사 할인 전혀 장바구니 할...,[구매]
1,김오성,2024.11.11,5,우주오란다 플래인 강정 주문햇습니다 이거 반틈은 내가 먹고 반틈은 울아부지 드릴껍니...,맛있어요,우주 래인 강정 햇습니 아부지 드릴 아부지 우주 래인 강정 옛날 이제 나이 무가 요...,None
2,김오성,2024.09.26,5,ㅈㅣ는 84세이신 울아부지랑 둘이서 살고 있습니다 울아부지 옛날에는 장승같이 커다란...,맛있어요,아부지 둘이서 아부지 옛날 장승 모습 는데 나이 가부장 잘못 뚜디리 기억 아부지 아...,[아부]
3,이*진,2024.09.22,4,《 우주오란다 》 * 용량 : 30g × 12개 * 소비기한 : 2025.04...,맛있어요,우주 용량 소비 기한 구매 이유 물엿 코팅 달달 식감 과자 특유 냄새 추억 우주 쿠...,"[포장, 식감, 우주]"
4,36참나리,2024.09.22,5,"클래식한 간식 중 최고봉, 오란다.달달한 간식이 생각나 주문했다.옛날 문방구에서 사...",맛있어요,클래식 간식 최고봉 달달 간식 주문 옛날 문방구 오란 입천장 이번 구매 수제 강정 ...,"[네덜란, 오란]"


# Model2: KEYBERT 이용

In [6]:
from keybert import KeyBERT

model = KeyBERT()
results = []

In [7]:


# for review in reviews:
#     keywords = model.extract_keywords(review, keyphrase_ngram_range=(1, 2), stop_words=None, top_n=3)     
#     results.append({'review': review,
#                     'topics': [kw[0] for kw in keywords]
#                     })
# result_df = pd.DataFrame(results)
# result_df

In [9]:
# results_doc = []
# keywords = model.extract_keywords(doc, keyphrase_ngram_range=(1, 2), stop_words=None, top_n=3)     
# results_doc.append({'review': doc,
#                     'topics': [kw[0] for kw in keywords]})

In [10]:
# results_doc

성능이 가히 좋아보이지 않음 -> 특정 키워드로 재매핑하고 결과를 보내야할지 고민